In [1]:
#feedback: reproducibility를 위해 randomness 통제를 위한 다음의 코드 추가하면 좋을 것 같습니다. (해당 시드는 고정 후, 아래 TrainingArguments의 seed 만 0-9까지 바꾸는 식으로 실험 진행하시면 될 것 같습니다.)
import numpy as np
import random
import torch
np.random.seed(0)
random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)

In [2]:
!git clone https://github.com/princeton-nlp/SimCSE.git

Cloning into 'SimCSE'...
remote: Enumerating objects: 434, done.
remote: Total 434 (delta 0), reused 0 (delta 0), pack-reused 434
Receiving objects: 100% (434/434), 41.46 MiB | 13.35 MiB/s, done.
Resolving deltas: 100% (209/209), done.


In [3]:
%cd SimCSE
!python setup.py install

/content/SimCSE
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other s

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [4]:
! pip install datasets
!pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 21.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 98.6 MB/s eta 0:00

In [5]:
from simcse import SimCSE
model = SimCSE("princeton-nlp/sup-simcse-bert-base-uncased")

In [6]:
# feedback: notebook_login()은 현재로서는 필요하지 않은 것 같습니다.
from huggingface_hub import notebook_login

notebook_login()

In [7]:
import transformers

print(transformers.__version__)

4.28.0


In [8]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

SST-2 (Stanford Sentiment Treebank) Determine if the sentence has a positive or negative sentiment.
`label (class label)에서 negative:0 positive:1`

In [9]:
task = "sst2"
model_checkpoint = "bert-base-uncased"
batch_size = 10

# Loading the dataset

In [10]:
from datasets import load_dataset, load_metric, concatenate_datasets

In [11]:

actual_task = "sst2"
dataset = load_dataset("glue", actual_task) #데이터셋 불러오기
metric = load_metric('glue', actual_task)


train_data = dataset['train']
val_data = dataset['validation']
test_data = dataset['test']   


# sst2는 train, validation 합치지 않고 train_data만 사용
# combined_data = concatenate_datasets([train_data, val_data])
combined_data = train_data
# 예들들어, 여기서 무작위 10개 뽑아서(데이터축소) train-dev 비율 나눠 측정

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

<ipython-input-11-8dc89cc9be2a>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', actual_task)


In [12]:
dataset #train, dev, test 데이터셋

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [13]:
combined_data #train 과 val 을 합친 데이터셋, *주의할점* train, dev를 합쳐서 class마다 n개를 뽑아야함

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 67349
})

In [14]:
#class(label) 마다 n개씩 추출하기 위해서 class마다 수 확인하기, positive:1, negative:0
positive = 0
negative = 0

for i in range(0, len(combined_data)):
  if combined_data[i]['label'] == 0:
    negative += 1
  else: positive += 1

print(positive, negative)


37569 29780


In [15]:
# Dataset.filter() 함수를 사용하여 레이블이 positive인 데이터와 negative인 데이터를 각각 필터링
positive_data = combined_data.filter(lambda example: example['label'] == 1)
negative_data = combined_data.filter(lambda example: example['label'] == 0)

Filter:   0%|          | 0/67349 [00:00<?, ? examples/s]

Filter:   0%|          | 0/67349 [00:00<?, ? examples/s]

In [16]:
#다음으로, positive_data와 negative_data에서 각각 무작위로 10개씩 데이터를 추출. 
#이를 위해서는 Dataset.shuffle() 함수를 사용
#여기서 shuffle() 함수는 데이터를 무작위로 섞는 역할을 하고, select() 함수는 추출할 데이터의 인덱스를 지정.
positive_data = positive_data.shuffle(seed=2).select(range(100)) #변경필요부분
negative_data = negative_data.shuffle(seed=2).select(range(100)) #변경필요부분
 

#이 코드를 실행하면, positive_data와 negative_data에서 각각 무작위로 10개씩 데이터가 추출.
print(positive_data)
print(negative_data)

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 100
})
Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 100
})


In [17]:
#무작위로 추출되는지 확인하기 위한 코드 [169]코드를 돌리고 이 코드를 돌려야 랜덤하게 10개씩 출력됨.
print(positive_data['sentence'])
print(negative_data['sentence'])

['that watching it leaves you giddy ', 'is a strength of a documentary to disregard available bias , especially ', 'the film is a pleasant enough dish . ', 'introduces you to new , fervently held ideas and fanciful thinkers ', 'sweet spirit ', 'returns as a visionary with a tale full of nuance and character dimension ', 'are immune to the folly of changing taste and attitude ', 'thanks to the presence of ` the king ', 'effecting ', 'is its surreal sense of humor and technological finish ', 'a rollicking adventure for you ', 'that encompasses many more paths than we started with ', 'good chance ', 'banter-filled comedy ', "when it 's this rich and luscious ", 'has created a breathtakingly assured and stylish work of spare dialogue and acute expressiveness . ', 'hero ', ', very good ', 'gets better after foster leaves that little room . ', 'one of the most genuinely sweet films to come along in quite some time ', 'absolutely ', 'is a fun and funky look into an artificial creation in a wo

In [18]:
#positive_data와 negative_data를 합친 새로운 데이터셋
# sampled_data = concatenate_datasets([positive_data, negative_data])
# print(sampled_data)


In [19]:
num_of_dev = 2

In [20]:
def find_second(L):
  first = second = 0
  first_idx = second_idx = 0
  for i in range(0, len(L)):
    if L[i] > first:
        second = first
        second_idx = first_idx
        first = L[i]
        first_idx = i
    elif second < L[i] < first:
        second = L[i]
        second_idx = i
  return second_idx

In [21]:
simil = []

for i in range(0, len(positive_data['sentence'])):
    simil.append(i)
    simil[i] = []
    temp = []
    for j in range(0, len(positive_data['sentence'])):
      similarities = model.similarity(positive_data['sentence'][i], positive_data['sentence'][j])
      temp.append(similarities)
    simil[i] = temp


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
100%|██████████| 1/1 [00:00<00:00, 59.09it/s]


In [22]:
done = []
train = []
dev = []

for i in range(0, len(simil[i])):
  if len(dev) == 10*num_of_dev:
    break
  if i not in done:
    idx = find_second(simil[i])
    if idx not in done:
          train.append(positive_data['idx'][i])
          dev.append(positive_data['idx'][idx])
          done.append(i)
          done.append(idx)

for i in range(0, len(simil[i])):
  if i not in done:
    train.append(positive_data['idx'][i])
  

In [23]:
simil = []

for i in range(0, len(negative_data['sentence'])):
    simil.append(i)
    simil[i] = []
    temp = []
    for j in range(0, len(negative_data['sentence'])):
      similarities = model.similarity(negative_data['sentence'][i], negative_data['sentence'][j])
      temp.append(similarities)
    simil[i] = temp

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
100%|██████████| 1/1 [00:00<00:00, 50.01it/s]


In [24]:
done = []

for i in range(0, len(simil[i])):
  if len(dev) == 20*num_of_dev:
    break
  if i not in done:
    idx = find_second(simil[i])
    if idx not in done:
          train.append(negative_data['idx'][i])
          dev.append(negative_data['idx'][idx])
          done.append(i)
          done.append(idx)

for i in range(0, len(simil[i])):
  if i not in done:
    train.append(negative_data['idx'][i])

In [25]:
from sklearn.model_selection import train_test_split
from datasets import DatasetDict

new_dataset = DatasetDict({'train': combined_data.select(train), 
                           'validation': combined_data.select(dev),
                           'test': val_data})

In [26]:
new_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 160
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 40
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
})

In [27]:
# # sampled_data와 test_data를 따로 분리해서 새로운 데이터셋을 만들기 위해 DatasetDict를 사용
# from sklearn.model_selection import train_test_split
# from datasets import DatasetDict
# # train_test_split() 함수는 데이터를 무작위로 섞어주고, 지정한 test_size 비율에 따라 데이터를 분할
# # sampled_data를 train과 validation(dev) 데이터셋으로 나누기 위해 train_test_split() 함수를 사용
# train_dev = sampled_data.train_test_split(shuffle = True, seed = 0, test_size=0.6)  #변경필요부분

# #train_dev와 test_data를 합쳐서 새로운 DatasetDict을 만들기
# #새로운 new_dataset은 train, validation, test의 key를 가지고 
# #각각 해당하는 데이터셋을 value로 가지는 DatasetDict
# new_dataset = DatasetDict({'train': train_dev['train'], 
#                            'validation': train_dev['test'],
#                            'test': val_data})

# print(new_dataset)

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [28]:
#Hugging Face의 Datasets 라이브러리를 사용하여 데이터셋에서 
#무작위로 num_examples개의 데이터를 선택하고, Pandas DataFrame을 사용하여 이를 표시하는 함수
import datasets
# import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = [] # 리스트를 초기화
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick) #선택된 인덱스를 picks 리스트에 추가합니다.

    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [29]:
#sample_data = show_random_elements(combined_data) #합쳐진 데이터에서 10개를 무작위로 추출한 데이터
#여기서 train-val 비율을 나눠 학습 시킨다.

The metric is an instance of datasets.Metric:

In [30]:
metric

Metric(name: "glue", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = datasets.load_metric('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(res

In [31]:
# import numpy as np

fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)

{'accuracy': 0.46875}

# Preprocessing the data

In [32]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True, )

In [33]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [34]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

In [35]:
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

Sentence: hide new secretions from the parental units 


In [36]:
def preprocess_function(examples):
    if sentence2_key is None:
        # return tokenizer(examples[sentence1_key], truncation=True)
        return tokenizer(examples[sentence1_key], padding='max_length', max_length=32 , truncation=True) #feedback:  padding과 max_length 설정을 하는 것이 좋을 것 같습니다. padding 옵션은 그대로 두시고, dataset 마다 max_legnth는 그에 맞게 수정하시면 좋을 것 같습니다.
    # return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], padding='max_length', max_length=32 , truncation=True)

In [37]:
preprocess_function(dataset['train'][:5])

{'input_ids': [[101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 3397, 2053, 15966, 1010, 2069, 4450, 2098, 18201, 2015, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2008, 7459, 2049, 3494, 1998, 10639, 2015, 2242, 2738, 3376, 2055, 2529, 3267, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 3464, 12580, 8510, 2000, 3961, 1996, 2168, 2802, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2006, 1996, 5409, 7195, 1011, 1997, 1011, 1996, 1011, 11265, 17811, 18856, 17322, 2015, 1996, 16587, 2071, 2852, 24225, 2039, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [38]:
encoded_dataset = new_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

# Fine-tuning the model

In [39]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [40]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

# seed 0

In [41]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-5, #feedback: learning rate은 이전 논문들의 hyperparamter를 참고하여 1e-5로 고정하여 실험하는 것이 좋을 것 같습니다.
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=30, #feedback: epoch은 이전 논문의 hyperparamter를 참고하여 30으로 고정하여 실험하는 것이 좋을 것 같습니다.
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    seed = 0, # seed 변경필요부분 0~
)

In [42]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [43]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.669756,0.575000
2,No log,0.638181,0.575000
3,No log,0.567444,0.750000
4,No log,0.477236,0.800000
5,No log,0.387875,0.850000
6,No log,0.349593,0.825000
7,No log,0.416305,0.825000
8,No log,0.470775,0.825000
9,No log,0.429445,0.875000
10,No log,0.553472,0.825000


TrainOutput(global_step=480, training_loss=0.10856982072194417, metrics={'train_runtime': 309.4328, 'train_samples_per_second': 15.512, 'train_steps_per_second': 1.551, 'total_flos': 78933316608000.0, 'train_loss': 0.10856982072194417, 'epoch': 30.0})

In [44]:
trainer.evaluate()

{'eval_loss': 0.42944544553756714,
 'eval_accuracy': 0.875,
 'eval_runtime': 0.1519,
 'eval_samples_per_second': 263.257,
 'eval_steps_per_second': 26.326,
 'epoch': 30.0}

In [45]:
#feedback: 최종적으로 각각의 (n, split비율)마다 seed 0-9까지의 다음 accuracy의 평균을 report 하시면 좋을 것 같습니다.
trainer.predict(encoded_dataset["test"])

PredictionOutput(predictions=array([[-1.4954    ,  2.495105  ],
       [ 1.3041298 , -1.9304156 ],
       [-1.613517  ,  2.2866747 ],
       ...,
       [ 0.9197497 , -1.6004978 ],
       [ 0.97529805, -1.5974864 ],
       [-0.2947704 ,  0.4265312 ]], dtype=float32), label_ids=array([1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0

# seed 1

In [46]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-5, #feedback: learning rate은 이전 논문들의 hyperparamter를 참고하여 1e-5로 고정하여 실험하는 것이 좋을 것 같습니다.
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=30, #feedback: epoch은 이전 논문의 hyperparamter를 참고하여 30으로 고정하여 실험하는 것이 좋을 것 같습니다.
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    seed = 1, # seed 변경필요부분 0~
)

In [47]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [48]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.512932,0.875000
2,No log,0.681166,0.850000
3,No log,0.735202,0.850000
4,No log,0.802244,0.825000
5,No log,1.028474,0.825000
6,No log,1.011219,0.825000
7,No log,0.934202,0.825000
8,No log,0.994980,0.825000
9,No log,1.037063,0.825000
10,No log,1.091384,0.825000


TrainOutput(global_step=480, training_loss=0.0013542627294858298, metrics={'train_runtime': 769.9801, 'train_samples_per_second': 6.234, 'train_steps_per_second': 0.623, 'total_flos': 78933316608000.0, 'train_loss': 0.0013542627294858298, 'epoch': 30.0})

In [49]:
trainer.evaluate()

{'eval_loss': 0.5129319429397583,
 'eval_accuracy': 0.875,
 'eval_runtime': 0.1476,
 'eval_samples_per_second': 271.09,
 'eval_steps_per_second': 27.109,
 'epoch': 30.0}

In [50]:
#feedback: 최종적으로 각각의 (n, split비율)마다 seed 0-9까지의 다음 accuracy의 평균을 report 하시면 좋을 것 같습니다.
trainer.predict(encoded_dataset["test"])

PredictionOutput(predictions=array([[-1.9893337 ,  2.9134047 ],
       [ 1.7928032 , -2.5262702 ],
       [-2.088885  ,  2.9258213 ],
       ...,
       [ 1.7055597 , -2.5421023 ],
       [ 1.5749489 , -2.4258795 ],
       [-0.5358351 ,  0.83778346]], dtype=float32), label_ids=array([1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0

# seed 2

In [51]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-5, #feedback: learning rate은 이전 논문들의 hyperparamter를 참고하여 1e-5로 고정하여 실험하는 것이 좋을 것 같습니다.
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=30, #feedback: epoch은 이전 논문의 hyperparamter를 참고하여 30으로 고정하여 실험하는 것이 좋을 것 같습니다.
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    seed = 2, # seed 변경필요부분 0~
)

In [52]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [53]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.660118,0.875000
2,No log,0.752134,0.850000
3,No log,1.015139,0.825000
4,No log,1.115313,0.825000
5,No log,1.071030,0.850000
6,No log,1.206991,0.825000
7,No log,1.252371,0.825000
8,No log,1.282851,0.825000
9,No log,1.303748,0.825000
10,No log,1.320596,0.825000


TrainOutput(global_step=480, training_loss=0.0010753276447455088, metrics={'train_runtime': 955.7207, 'train_samples_per_second': 5.022, 'train_steps_per_second': 0.502, 'total_flos': 78933316608000.0, 'train_loss': 0.0010753276447455088, 'epoch': 30.0})

In [54]:
trainer.evaluate()

{'eval_loss': 0.6601181030273438,
 'eval_accuracy': 0.875,
 'eval_runtime': 0.1325,
 'eval_samples_per_second': 301.948,
 'eval_steps_per_second': 30.195,
 'epoch': 30.0}

In [55]:
#feedback: 최종적으로 각각의 (n, split비율)마다 seed 0-9까지의 다음 accuracy의 평균을 report 하시면 좋을 것 같습니다.
trainer.predict(encoded_dataset["test"])

PredictionOutput(predictions=array([[-2.4230027 ,  3.2158144 ],
       [ 2.4091527 , -3.2881463 ],
       [-2.346169  ,  3.097666  ],
       ...,
       [ 2.207406  , -3.110606  ],
       [ 2.2137413 , -3.1097517 ],
       [ 0.05940981,  0.02408032]], dtype=float32), label_ids=array([1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0

# seed 3

In [56]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-5, #feedback: learning rate은 이전 논문들의 hyperparamter를 참고하여 1e-5로 고정하여 실험하는 것이 좋을 것 같습니다.
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=30, #feedback: epoch은 이전 논문의 hyperparamter를 참고하여 30으로 고정하여 실험하는 것이 좋을 것 같습니다.
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    seed = 3, # seed 변경필요부분 0~
)

In [57]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [58]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.935821,0.825000
2,No log,0.717161,0.875000
3,No log,0.882664,0.850000
4,No log,1.050268,0.850000
5,No log,1.130970,0.825000
6,No log,1.114898,0.850000
7,No log,1.126649,0.850000
8,No log,0.861551,0.875000
9,No log,0.883578,0.900000
10,No log,1.284011,0.850000


TrainOutput(global_step=480, training_loss=0.0025410227477550506, metrics={'train_runtime': 975.7021, 'train_samples_per_second': 4.92, 'train_steps_per_second': 0.492, 'total_flos': 78933316608000.0, 'train_loss': 0.0025410227477550506, 'epoch': 30.0})

In [59]:
trainer.evaluate()

{'eval_loss': 0.8835777044296265,
 'eval_accuracy': 0.9,
 'eval_runtime': 0.1526,
 'eval_samples_per_second': 262.132,
 'eval_steps_per_second': 26.213,
 'epoch': 30.0}

In [60]:
#feedback: 최종적으로 각각의 (n, split비율)마다 seed 0-9까지의 다음 accuracy의 평균을 report 하시면 좋을 것 같습니다.
trainer.predict(encoded_dataset["test"])

PredictionOutput(predictions=array([[-4.057798 ,  4.8311954],
       [ 4.414037 , -4.9301953],
       [-3.9758203,  4.5806556],
       ...,
       [ 4.3388853, -4.9136515],
       [ 4.2555256, -4.700566 ],
       [ 3.7598844, -4.103651 ]], dtype=float32), label_ids=array([1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1,
    

# seed 4

In [61]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-5, #feedback: learning rate은 이전 논문들의 hyperparamter를 참고하여 1e-5로 고정하여 실험하는 것이 좋을 것 같습니다.
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=30, #feedback: epoch은 이전 논문의 hyperparamter를 참고하여 30으로 고정하여 실험하는 것이 좋을 것 같습니다.
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    seed = 4, # seed 변경필요부분 0~
)

In [62]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [63]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.295847,0.825000
2,No log,1.428347,0.850000
3,No log,1.420891,0.825000
4,No log,0.866075,0.875000
5,No log,0.834873,0.925000
6,No log,0.939621,0.875000
7,No log,1.304888,0.850000
8,No log,1.483829,0.850000
9,No log,1.517404,0.825000
10,No log,1.535280,0.825000


TrainOutput(global_step=480, training_loss=0.003611317276954651, metrics={'train_runtime': 685.5612, 'train_samples_per_second': 7.002, 'train_steps_per_second': 0.7, 'total_flos': 78933316608000.0, 'train_loss': 0.003611317276954651, 'epoch': 30.0})

In [64]:
trainer.evaluate()

{'eval_loss': 0.8348734974861145,
 'eval_accuracy': 0.925,
 'eval_runtime': 0.152,
 'eval_samples_per_second': 263.115,
 'eval_steps_per_second': 26.312,
 'epoch': 30.0}

In [65]:
#feedback: 최종적으로 각각의 (n, split비율)마다 seed 0-9까지의 다음 accuracy의 평균을 report 하시면 좋을 것 같습니다.
trainer.predict(encoded_dataset["test"])

PredictionOutput(predictions=array([[-4.8812175,  5.484347 ],
       [ 5.2529716, -5.63331  ],
       [-4.44492  ,  4.8882084],
       ...,
       [ 5.1136246, -5.592073 ],
       [ 5.251524 , -5.552648 ],
       [ 4.7109723, -5.088966 ]], dtype=float32), label_ids=array([1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1,
    

# seed 5

In [66]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-5, #feedback: learning rate은 이전 논문들의 hyperparamter를 참고하여 1e-5로 고정하여 실험하는 것이 좋을 것 같습니다.
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=30, #feedback: epoch은 이전 논문의 hyperparamter를 참고하여 30으로 고정하여 실험하는 것이 좋을 것 같습니다.
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    seed = 5, # seed 변경필요부분 0~
)

In [67]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [68]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.838972,0.825000
2,No log,1.952448,0.825000
3,No log,1.241223,0.875000
4,No log,1.371020,0.875000
5,No log,1.664543,0.850000
6,No log,1.657257,0.850000
7,No log,1.657857,0.825000
8,No log,1.705001,0.825000
9,No log,1.775781,0.850000
10,No log,1.742993,0.850000


TrainOutput(global_step=480, training_loss=6.186084647197276e-06, metrics={'train_runtime': 798.2929, 'train_samples_per_second': 6.013, 'train_steps_per_second': 0.601, 'total_flos': 78933316608000.0, 'train_loss': 6.186084647197276e-06, 'epoch': 30.0})

In [69]:
trainer.evaluate()

{'eval_loss': 1.2412234544754028,
 'eval_accuracy': 0.875,
 'eval_runtime': 0.1342,
 'eval_samples_per_second': 298.1,
 'eval_steps_per_second': 29.81,
 'epoch': 30.0}

In [70]:
#feedback: 최종적으로 각각의 (n, split비율)마다 seed 0-9까지의 다음 accuracy의 평균을 report 하시면 좋을 것 같습니다.
trainer.predict(encoded_dataset["test"])

PredictionOutput(predictions=array([[-5.609874 ,  6.1161942],
       [ 5.999421 , -6.2763515],
       [-5.527628 ,  5.8587737],
       ...,
       [ 5.924309 , -6.268043 ],
       [ 5.9672914, -6.192863 ],
       [ 5.5594873, -5.8294144]], dtype=float32), label_ids=array([1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1,
    

# seed 6

In [71]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-5, #feedback: learning rate은 이전 논문들의 hyperparamter를 참고하여 1e-5로 고정하여 실험하는 것이 좋을 것 같습니다.
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=30, #feedback: epoch은 이전 논문의 hyperparamter를 참고하여 30으로 고정하여 실험하는 것이 좋을 것 같습니다.
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    seed = 6, # seed 변경필요부분 0~
)

In [72]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [73]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.897794,0.825000
2,No log,2.024004,0.800000
3,No log,1.974670,0.825000
4,No log,1.820778,0.825000
5,No log,1.765926,0.825000
6,No log,1.207240,0.900000
7,No log,1.530672,0.850000
8,No log,1.810183,0.775000
9,No log,1.501878,0.800000
10,No log,1.332010,0.850000


TrainOutput(global_step=480, training_loss=0.00465087741613388, metrics={'train_runtime': 812.7751, 'train_samples_per_second': 5.906, 'train_steps_per_second': 0.591, 'total_flos': 78933316608000.0, 'train_loss': 0.00465087741613388, 'epoch': 30.0})

In [74]:
trainer.evaluate()

{'eval_loss': 1.207240104675293,
 'eval_accuracy': 0.9,
 'eval_runtime': 0.136,
 'eval_samples_per_second': 294.139,
 'eval_steps_per_second': 29.414,
 'epoch': 30.0}

In [75]:
#feedback: 최종적으로 각각의 (n, split비율)마다 seed 0-9까지의 다음 accuracy의 평균을 report 하시면 좋을 것 같습니다.
trainer.predict(encoded_dataset["test"])

PredictionOutput(predictions=array([[-5.987336 ,  6.449244 ],
       [ 6.3096566, -6.544009 ],
       [-5.8008466,  6.095397 ],
       ...,
       [ 6.208717 , -6.5171223],
       [ 6.3038945, -6.4707823],
       [ 5.697019 , -5.8678513]], dtype=float32), label_ids=array([1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1,
    

# seed 7

In [76]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-5, #feedback: learning rate은 이전 논문들의 hyperparamter를 참고하여 1e-5로 고정하여 실험하는 것이 좋을 것 같습니다.
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=30, #feedback: epoch은 이전 논문의 hyperparamter를 참고하여 30으로 고정하여 실험하는 것이 좋을 것 같습니다.
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    seed = 7, # seed 변경필요부분 0~
)

In [77]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.202683,0.900000
2,No log,1.658400,0.850000
3,No log,1.814513,0.850000
4,No log,1.894161,0.850000
5,No log,1.922448,0.850000
6,No log,1.955021,0.850000
7,No log,1.984801,0.850000
8,No log,2.007733,0.850000
9,No log,2.016445,0.850000
10,No log,2.027227,0.850000


TrainOutput(global_step=480, training_loss=8.401769567475034e-07, metrics={'train_runtime': 791.3063, 'train_samples_per_second': 6.066, 'train_steps_per_second': 0.607, 'total_flos': 78933316608000.0, 'train_loss': 8.401769567475034e-07, 'epoch': 30.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 1.2026828527450562,
 'eval_accuracy': 0.9,
 'eval_runtime': 0.1321,
 'eval_samples_per_second': 302.697,
 'eval_steps_per_second': 30.27,
 'epoch': 30.0}

In [ ]:
#feedback: 최종적으로 각각의 (n, split비율)마다 seed 0-9까지의 다음 accuracy의 평균을 report 하시면 좋을 것 같습니다.
trainer.predict(encoded_dataset["test"])

PredictionOutput(predictions=array([[-6.2217546,  6.649492 ],
       [ 6.210903 , -6.419961 ],
       [-6.1360083,  6.4205117],
       ...,
       [ 6.3699613, -6.635403 ],
       [ 6.2874656, -6.397241 ],
       [ 4.24868  , -4.17541  ]], dtype=float32), label_ids=array([1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1,
    

# seed 8

In [ ]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-5, #feedback: learning rate은 이전 논문들의 hyperparamter를 참고하여 1e-5로 고정하여 실험하는 것이 좋을 것 같습니다.
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=30, #feedback: epoch은 이전 논문의 hyperparamter를 참고하여 30으로 고정하여 실험하는 것이 좋을 것 같습니다.
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    seed = 8, # seed 변경필요부분 0~
)

In [ ]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.576084,0.850000
2,No log,1.761052,0.850000
3,No log,1.555395,0.875000
4,No log,1.600284,0.875000
5,No log,1.644525,0.875000
6,No log,1.683293,0.850000
7,No log,1.731397,0.850000
8,No log,1.774565,0.850000
9,No log,1.821308,0.850000
10,No log,1.851153,0.850000


TrainOutput(global_step=480, training_loss=6.840626762520212e-07, metrics={'train_runtime': 721.155, 'train_samples_per_second': 6.656, 'train_steps_per_second': 0.666, 'total_flos': 78933316608000.0, 'train_loss': 6.840626762520212e-07, 'epoch': 30.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 1.5553946495056152,
 'eval_accuracy': 0.875,
 'eval_runtime': 0.1357,
 'eval_samples_per_second': 294.876,
 'eval_steps_per_second': 29.488,
 'epoch': 30.0}

In [ ]:
#feedback: 최종적으로 각각의 (n, split비율)마다 seed 0-9까지의 다음 accuracy의 평균을 report 하시면 좋을 것 같습니다.
trainer.predict(encoded_dataset["test"])

PredictionOutput(predictions=array([[-6.719503  ,  7.075171  ],
       [ 6.042916  , -6.294467  ],
       [-6.635293  ,  6.879201  ],
       ...,
       [ 6.7957206 , -7.0416174 ],
       [ 6.567025  , -6.650752  ],
       [-0.12209654,  0.43281513]], dtype=float32), label_ids=array([1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0

# seed 9

In [ ]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-5, #feedback: learning rate은 이전 논문들의 hyperparamter를 참고하여 1e-5로 고정하여 실험하는 것이 좋을 것 같습니다.
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=30, #feedback: epoch은 이전 논문의 hyperparamter를 참고하여 30으로 고정하여 실험하는 것이 좋을 것 같습니다.
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    seed = 9, # seed 변경필요부분 0~
)

In [ ]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.662965,0.850000
2,No log,1.790042,0.850000
3,No log,1.854045,0.850000
4,No log,1.881585,0.850000
5,No log,1.928255,0.850000
6,No log,1.952825,0.850000
7,No log,1.958294,0.850000
8,No log,1.978242,0.850000
9,No log,1.993243,0.850000
10,No log,2.007676,0.850000


TrainOutput(global_step=480, training_loss=3.5713104201325526e-07, metrics={'train_runtime': 777.7346, 'train_samples_per_second': 6.172, 'train_steps_per_second': 0.617, 'total_flos': 78933316608000.0, 'train_loss': 3.5713104201325526e-07, 'epoch': 30.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 1.6629654169082642,
 'eval_accuracy': 0.85,
 'eval_runtime': 0.1393,
 'eval_samples_per_second': 287.164,
 'eval_steps_per_second': 28.716,
 'epoch': 30.0}

In [ ]:
#feedback: 최종적으로 각각의 (n, split비율)마다 seed 0-9까지의 다음 accuracy의 평균을 report 하시면 좋을 것 같습니다.
trainer.predict(encoded_dataset["test"])

PredictionOutput(predictions=array([[-6.8363404,  7.1772466],
       [ 5.815699 , -6.0335073],
       [-6.781596 ,  7.0157866],
       ...,
       [ 6.9124255, -7.1363125],
       [ 6.5372443, -6.595762 ],
       [-1.1713082,  1.5382903]], dtype=float32), label_ids=array([1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1,
    